# Dynamic Tiling

This notebook explores creating a series dynamicaly sized tiles out of a vector (rank-1 tensor). 

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=tree --animation=movie

# Dynamic tiling class

The following class is a generic class useful for tiling based on a computed criteria


In [ ]:
class DynamicTiler():
    """ DymamicTile
    
    A callable class usable by the Fiber.prune() method to
    create a tile based on a given lambda function that 
    decides if each element belongs in the tile
    
    """
    def __init__(self, include_in_tile, max_tile_size=1):
        """ __init__()
        
        Initialize the class

        Parameters
        -----------
        include_in_tile: function(coord, payload, cur_tile_size)
        Function called to decide if each element should be in the tile
        
        max_tile_size: integer
        Maximum tile size
        
        """
        
        self.include_in_tile = include_in_tile
        self.max_tile_size = max_tile_size
        self.cur_tile_size = 0


    def __call__(self, n, c, p):
        """ __call__()
        
        Function callable by Fiber.prune() to decide what to do
        with each element of a fiber. Either keep the element or
        end the tile
        
        Parameters:
        
        n: integer
        Position of element in tile
        
        c: coordinate
        Coordinate of element in tile
        
        p: payload
        Payload of elenent in tile
        
        Return
        ------
        
        Prune: Boolean | None
               True - Include
               False - Skip (never used)
               None - End tile
               
        """
        #
        # Terminate the tile if it is already big enough
        #
        if self.cur_tile_size >= self.max_tile_size:
            self.cur_tile_size = 0
            return None
        
        #
        # Ask if we want another element in the tile
        #
        include = self.include_in_tile(c, p, self.cur_tile_size)
        
        #
        # Yep, add element to tile and increment the tile size
        #
        if include == True:
            self.cur_tile_size += 1
            return True

        #
        # End the tile
        #
        self.cur_tile_size = 0
        return None
    

## Element selection function

A function that randomly determines if an element of the original tensor is to be included in the current tile.

In [ ]:
        
import random

def random_tile(c, p, size):
    """ random_tile
    
    Decide if an element of a fiber should be included in
    the current tile
    
    Parameters
    ----------
    
    c: coordinate
    The coordinate of the element to consider for inclusion
    
    p: payload
    The payload of the element to conder for inclusion
    
    size: integer
    The number of elements already in the current tile
    
    Return
    -------
    
    Include: Boolean
    True means include this element; False means end tile
    
    """
    #
    # Tiles must have a least one element, but we
    # get called in case this method has some internal
    # state that gets updated
    #
    if size == 0:
        return True
    
    #
    # Randomly decide if we want more elements
    #
    return random.random() > .2

## Dynamically tile a tensor

In [ ]:
t = Tensor.fromRandom(["X"], [40], (0.6,), 10, seed=40)
t.setColor("green").setName("T")

displayTensor(t)
t_x1 = t.getRoot()

canvas = createCanvas(t)

tile_filter = DynamicTiler(random_tile, 4)

start_pos = 0

while start_pos < len(t_x1):
    print(f"Tile starting at position: {start_pos}")
    
    t_x0 = t_x1.prune(tile_filter, start_pos=start_pos)
    start_pos = t_x1.getSavedPos()

    # displayTensor(t_x0)
    addFrame(canvas, [(c,) for c,p in t_x0])

    for c, p in t_x0:
        print(f"c: {c}, p: {p}")


(count, distance) = t_x1.getSavedPosStats()
print("")
print(f"Average search distance: {distance/count:4.2}")
        
displayCanvas(canvas)


## Testing area

For running alternative algorithms